In [4]:
# 1. Universal QEPC header + nbainjuries imports

import os
import sys
from pathlib import Path

# Try to import notebook_context directly
try:
    from notebook_context import *  # sets project_root, paths, etc.
    print("✅ Imported notebook_context directly.")
except ModuleNotFoundError:
    print("ℹ️ notebook_context not found on sys.path; trying to locate it...")

    cwd = Path.cwd()
    candidate_roots = [
        cwd,
        cwd.parent,
        cwd.parent.parent,
    ]

    found_root = None
    for root in candidate_roots:
        if (root / "notebook_context.py").exists():
            found_root = root
            break

    if found_root is None:
        raise ModuleNotFoundError(
            f"Could not find notebook_context.py in {cwd} or its parents. "
            f"Make sure this notebook lives under your qepc_project folder, "
            f"or open it from inside qepc_project."
        )

    # Add that root to sys.path and switch working directory to it
    sys.path.insert(0, str(found_root))
    os.chdir(found_root)
    print(f"🔗 Added {found_root} to sys.path and changed working directory there.")

    from notebook_context import *
    print("✅ Imported notebook_context after path adjustment.")

# Now pull in nbainjuries + friends
from nbainjuries import injury
from datetime import datetime
import pandas as pd

# Fallback for project_root if notebook_context didn't define it
try:
    project_root
except NameError:
    project_root = Path.cwd()
    print("⚠️ 'project_root' was not defined by notebook_context; "
          "using current working directory as project_root instead.")

print("Project root detected as:", project_root)
print("Current working directory:", os.getcwd())


✅ Imported notebook_context directly.
Project root detected as: C:\Users\wdors\qepc_project
Current working directory: C:\Users\wdors\qepc_project


In [5]:
# Cell 2: Fetch latest official injury report

dt_now = datetime.now()
print("Requesting official injury report for:", dt_now)

inj_raw_df = injury.get_reportdata(dt_now, return_df=True)

print("✅ Official injury report loaded.")
print("   Rows in raw official injury report:", len(inj_raw_df))
print("   Columns:", list(inj_raw_df.columns))

# If you ever want to visually inspect:
# display(inj_raw_df.head(10))


Requesting official injury report for: 2025-11-26 11:56:54.717218
Validated Injury-Report_2025-11-26_11AM.
Rows in raw official injury report: 88


,Game Date,Game Time,Matchup,Team,Player Name,Current Status,Reason
0,11/26/2025,05:00 (ET),DET@BOS,Detroit Pistons,"Jones, Isaac",Out,G League - On Assignment
1,11/26/2025,05:00 (ET),DET@BOS,Detroit Pistons,"Klintman, Bobi",Out,Injury/Illness - Left Ankle; Sprain
2,11/26/2025,05:00 (ET),DET@BOS,Detroit Pistons,"Moore Jr., Wendell",Out,G League - Two-Way
3,11/26/2025,05:00 (ET),DET@BOS,Detroit Pistons,"Sasser, Marcus",Out,Injury/Illness - Right Hip; Impingement
4,11/26/2025,05:00 (ET),DET@BOS,Detroit Pistons,"Smith, Tolu",Out,G League - Two-Way
5,11/26/2025,05:00 (ET),DET@BOS,Boston Celtics,"Harper Jr., Ron",Available,G League - Two-Way
6,11/26/2025,05:00 (ET),DET@BOS,Boston Celtics,"Queta, Neemias",Out,Injury/Illness - Left Ankle; Sprain
7,11/26/2025,05:00 (ET),DET@BOS,Boston Celtics,"Shulga, Max",Available,G League - Two-Way
8,11/26/2025,05:00 (ET),DET@BOS,Boston Celtics,"Tatum, Jayson",Out,Injury/Illness - Right Achilles; Repair
9,11/26/2025,07:00 (ET),NYK@CHA,New York Knicks,"Anunoby, OG",Out,Injury/Illness - Left Hamstring; Strain


In [6]:
# Cell 3: Convert to QEPC format and cache as CSVs

from datetime import datetime
import pandas as pd

def status_to_impact(status: str) -> float:
    """
    Simple impact scale based on the official status text.
    This is a 'live' layer – your data-driven injury CSV can still override/augment.
    """
    if not isinstance(status, str):
        return 0.95

    s = status.lower()

    if "out" in s:
        return 0.70   # big impact
    if "doubtful" in s:
        return 0.85
    if "questionable" in s:
        return 0.90
    if "probable" in s:
        return 0.95
    if "available" in s or "cleared" in s or "active" in s:
        return 1.00

    # default mild discount
    return 0.95


# Build QEPC-style injury table from the raw official report
inj_qepc = pd.DataFrame({
    "Team": inj_raw_df["Team"],
    "PlayerName": inj_raw_df["Player Name"],
    "Status": inj_raw_df["Current Status"],
    "Injury": inj_raw_df["Reason"],
    "EstReturn": "",  # official reports are game-by-game, not long-term ETA text
})

inj_qepc["Impact"] = inj_qepc["Status"].apply(status_to_impact)
inj_qepc["Source"] = "NBA_official_nbainjuries"

# Add a report timestamp column so we know which snapshot this is
run_ts = datetime.now()
inj_qepc["Report_Timestamp"] = run_ts.isoformat(timespec="seconds")

# Ensure the cache directory exists for versioned snapshots
cache_dir = project_root / "data" / "injury_cache"
cache_dir.mkdir(parents=True, exist_ok=True)

# 1) Rolling LATEST file (this is what QEPC reads everywhere)
latest_path = project_root / "data" / "Injury_Overrides_live_official.csv"
inj_qepc.to_csv(latest_path, index=False)

# 2) Versioned snapshot (for history / rollback / inspection)
stamp = run_ts.strftime("%Y-%m-%d_%H-%M-%S")
snapshot_path = cache_dir / f"Injury_Overrides_official_{stamp}.csv"
inj_qepc.to_csv(snapshot_path, index=False)

print("✅ QEPC injury overrides built.")
print("   Rows in QEPC injury override table:", len(inj_qepc))
print("   Saved LATEST file to:", latest_path)
print("   Saved SNAPSHOT file to:", snapshot_path)

# Optional one-line peek (commented out by default)
# display(inj_qepc.head(10))


Rows in QEPC injury override table: 88


,Team,PlayerName,Status,Injury,EstReturn,Impact,Source
0,Detroit Pistons,"Jones, Isaac",Out,G League - On Assignment,,0.7,NBA_official_nbainjuries
1,Detroit Pistons,"Klintman, Bobi",Out,Injury/Illness - Left Ankle; Sprain,,0.7,NBA_official_nbainjuries
2,Detroit Pistons,"Moore Jr., Wendell",Out,G League - Two-Way,,0.7,NBA_official_nbainjuries
3,Detroit Pistons,"Sasser, Marcus",Out,Injury/Illness - Right Hip; Impingement,,0.7,NBA_official_nbainjuries
4,Detroit Pistons,"Smith, Tolu",Out,G League - Two-Way,,0.7,NBA_official_nbainjuries
5,Boston Celtics,"Harper Jr., Ron",Available,G League - Two-Way,,1.0,NBA_official_nbainjuries
6,Boston Celtics,"Queta, Neemias",Out,Injury/Illness - Left Ankle; Sprain,,0.7,NBA_official_nbainjuries
7,Boston Celtics,"Shulga, Max",Available,G League - Two-Way,,1.0,NBA_official_nbainjuries
8,Boston Celtics,"Tatum, Jayson",Out,Injury/Illness - Right Achilles; Repair,,0.7,NBA_official_nbainjuries
9,New York Knicks,"Anunoby, OG",Out,Injury/Illness - Left Hamstring; Strain,,0.7,NBA_official_nbainjuries


In [7]:
from datetime import datetime
import pandas as pd

# --- Convert official report -> QEPC injury overrides + cache to disk ---

def status_to_impact(status: str) -> float:
    """
    Very simple impact scale based on the official status text.
    This is a 'live' layer – your data-driven injury CSV can still override/augment.
    """
    if not isinstance(status, str):
        return 0.95

    s = status.lower()

    if "out" in s:
        return 0.70   # big impact
    if "doubtful" in s:
        return 0.85
    if "questionable" in s:
        return 0.90
    if "probable" in s:
        return 0.95
    if "available" in s or "cleared" in s or "active" in s:
        return 1.00

    # default mild discount
    return 0.95


# Build QEPC-style injury table from the raw official report
inj_qepc = pd.DataFrame({
    "Team": inj_raw_df["Team"],
    "PlayerName": inj_raw_df["Player Name"],
    "Status": inj_raw_df["Current Status"],
    "Injury": inj_raw_df["Reason"],
    "EstReturn": "",  # official reports are game-by-game, not long-term ETA text
})

inj_qepc["Impact"] = inj_qepc["Status"].apply(status_to_impact)
inj_qepc["Source"] = "NBA_official_nbainjuries"

# Add a report timestamp column so we know which snapshot this is
run_ts = datetime.now()
inj_qepc["Report_Timestamp"] = run_ts.isoformat(timespec="seconds")

# Ensure the cache directory exists for versioned snapshots
cache_dir = project_root / "data" / "injury_cache"
cache_dir.mkdir(parents=True, exist_ok=True)

# 1) Rolling LATEST file (this is what QEPC reads everywhere)
latest_path = project_root / "data" / "Injury_Overrides_live_official.csv"
inj_qepc.to_csv(latest_path, index=False)
print("Saved LATEST official injury overrides to:", latest_path)

# 2) Versioned snapshot (for history / rollback / inspection)
stamp = run_ts.strftime("%Y-%m-%d_%H-%M-%S")
snapshot_path = cache_dir / f"Injury_Overrides_official_{stamp}.csv"
inj_qepc.to_csv(snapshot_path, index=False)
print("Saved SNAPSHOT injury overrides to:", snapshot_path)

print("Rows in QEPC injury override table:", len(inj_qepc))
display(inj_qepc.head(20))


Saved LATEST official injury overrides to: C:\Users\wdors\qepc_project\data\Injury_Overrides_live_official.csv
Saved SNAPSHOT injury overrides to: C:\Users\wdors\qepc_project\data\injury_cache\Injury_Overrides_official_2025-11-26_11-57-05.csv
Rows in QEPC injury override table: 88


,Team,PlayerName,Status,Injury,EstReturn,Impact,Source,Report_Timestamp
0,Detroit Pistons,"Jones, Isaac",Out,G League - On Assignment,,0.7,NBA_official_nbainjuries,2025-11-26T11:57:05
1,Detroit Pistons,"Klintman, Bobi",Out,Injury/Illness - Left Ankle; Sprain,,0.7,NBA_official_nbainjuries,2025-11-26T11:57:05
2,Detroit Pistons,"Moore Jr., Wendell",Out,G League - Two-Way,,0.7,NBA_official_nbainjuries,2025-11-26T11:57:05
3,Detroit Pistons,"Sasser, Marcus",Out,Injury/Illness - Right Hip; Impingement,,0.7,NBA_official_nbainjuries,2025-11-26T11:57:05
4,Detroit Pistons,"Smith, Tolu",Out,G League - Two-Way,,0.7,NBA_official_nbainjuries,2025-11-26T11:57:05
5,Boston Celtics,"Harper Jr., Ron",Available,G League - Two-Way,,1.0,NBA_official_nbainjuries,2025-11-26T11:57:05
6,Boston Celtics,"Queta, Neemias",Out,Injury/Illness - Left Ankle; Sprain,,0.7,NBA_official_nbainjuries,2025-11-26T11:57:05
7,Boston Celtics,"Shulga, Max",Available,G League - Two-Way,,1.0,NBA_official_nbainjuries,2025-11-26T11:57:05
8,Boston Celtics,"Tatum, Jayson",Out,Injury/Illness - Right Achilles; Repair,,0.7,NBA_official_nbainjuries,2025-11-26T11:57:05
9,New York Knicks,"Anunoby, OG",Out,Injury/Illness - Left Hamstring; Strain,,0.7,NBA_official_nbainjuries,2025-11-26T11:57:05


In [ ]:

]
